In [2]:
import ROOT
import os, sys, subprocess, time
import py8settings as py8s
from py8settings import PDF
import itertools as it
import ctypes

%jsroot on

Welcome to JupyROOT 6.24/04


In [9]:
def makePythiaSilent(pythia):
    pythia.ReadString("Init:showMultipartonInteractions = off")
    pythia.ReadString("Init:showChangedParticleData = off")
    pythia.ReadString("Init:showProcesses = off")
    pythia.ReadString("Init:showChangedSettings = off")
    pythia.ReadString("Next:numberShowInfo = 0")
    pythia.ReadString("Next:numberShowProcess = 0")
    pythia.ReadString("Next:numberShowEvent = 0")
    pythia.ReadString("Next:numberCount = 0")
    pythia.ReadString("WeakSingleBoson:ffbar2gmZ = on")
    pythia.ReadString("HadronLevel:all = off")      # maybe this is ok
    pythia.ReadString("23:onMode = off")    # disable decay of Z boson
    pythia.ReadString("23:onIfAny = 11 -11")    # enable decay of Z boson in channels containing electron and positon
    pythia.ReadString("PhaseSpace:mHatMin = 111")
    pythia.ReadString("PhaseSpace:mHatMax = 71")

pythia = ROOT.TPythia8(False)   # suppress the banner message
makePythiaSilent(pythia)
# pythia.ReadString("HardQCD:all = on")   # passing set of parameters, options in Pythia documentation; 
pythia.ReadString("Random:setSeed = on")
pythia.ReadString("Random:seed = 42") # to ro randomize seed; while we give taahe same seed we will receive the same results
pythia.Initialize(2212 , -2212 , 630)  # 2212 - proton in PDG codes, antiproton has a minus sign
particles = ROOT.TClonesArray("TParticle", 1000) #prepare space for generated particles

# Warning about cross section violated is ok

 *** TDatabasePDG::AddParticle: particle with PDGcode=90 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9900110 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9900210 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9900220 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9900330 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9900440 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9902110 already defined
 *** TDatabasePDG::AddParticle: particle with PDGcode=9902210 already defined


In [10]:
pythia.GenerateEvent()
pythia.EventListing()


 --------  PYTHIA Event Listing  (complete event)  ---------------------------------------------------------------------------------
 
    no         id  name            status     mothers   daughters     colours      p_x        p_y        p_z         e          m 
     0         90  (system)           -11     0     0     0     0     0     0      0.000      0.000      0.000    630.000    630.000
     1       2212  (p+)               -12     0     0   105     0     0     0      0.000      0.000    314.999    315.000      0.938
     2      -2212  (pbar-)            -12     0     0   106     0     0     0      0.000      0.000   -314.999    315.000      0.938
     3          2  (u)                -21     6     6     5     0   101     0      0.000      0.000     86.680     86.680      0.000
     4         -2  (ubar)             -21     7     0     5     0     0   101      0.000      0.000    -38.395     38.395      0.000
     5         23  (Z0)               -22     3     4     8     8   

In [11]:
def isFinal(p,name=None):
    if name==None:
        return p.GetStatusCode()>0
    else:
        return p.GetStatusCode()>0 and p.GetName()==name


def getMinv(pair):  # invariant mass of 2 particles
    p4_1=ROOT.TLorentzVector()  # ROOT class to represent 4vectors
    p4_2=ROOT.TLorentzVector()
    pair[0].Momentum(p4_1)
    pair[1].Momentum(p4_2)
    p4=p4_1+p4_2
    return p4.M()   # get mass of 4vector

In [12]:
nEvents=10000
parts=["electron", "antielectron", "gamma"]     # our particles
labels={"electron": "e-", "antielectron": "e+", "gamma" : "#gamma"}
colors={"proton": ROOT.kBlack, "antielectron": ROOT.kBlue, "gamma": ROOT.kMagenta}
heta={}
hpt={}

h=ROOT.TH1D("h_all","",300, 70, 110)    # histograms start
hpure=ROOT.TH1D("h_pure","",300, 70, 110)


def getZcandidates(parts):
    eminus = [p for p in parts if isFinal(p,"e-")]    # creates list
    eplus = [p for p in parts if isFinal(p,"e+")]
    return list(it.product(eminus,eplus))   # pair of electron and positon

pythia.GenerateEvent()

for i in range(nEvents):
    pythia.GenerateEvent()
    pythia.ImportParticles(particles,"All")
    #for all possible e- e+ pairs we fill invariant mass histogram
    for pair in getZcandidates(particles):
        # print("works")
        h.Fill(getMinv(pair))
    #now we look for actual Z's that decayed:
    for p in particles:
        if p.GetName()=="Z0":
            #we accept Z0 only if it decayed to electron and positon
            d0=particles[p.GetDaughter(0)].GetName()
            d1=particles[p.GetDaughter(1)].GetName()
            if {d0,d1} != {"e-","e+"}:     # curly braces represent set datatype in python
                continue
            p4=ROOT.TLorentzVector()
            p.Momentum(p4)
            hpure.Fill(p4.M())


c_kstar = ROOT.TCanvas("c","",800,600)
ROOT.gStyle.SetOptStat(0)
h.GetXaxis().SetTitle("M_{#e^{-}e^{+}} [GeV/#it{c}^{2}]")
h.GetXaxis().SetTitleOffset(1.1)
h.GetYaxis().SetTitle("Entries")
h.SetLineWidth(2)
h.Draw()
# hpure.SetLineWidth(2)
# hpure.SetLineColor(ROOT.kRed)
# hpure.Draw("same")
# leg=ROOT.TLegend(0.5,0.7,0.9,0.9)
# leg.AddEntry(h,"all #pi^{-}K^{+} pairs")
# leg.AddEntry(hpure,"#pi^{-}K^{+} pairs from K^{*} decay")
# leg.Draw()
c_kstar.Draw()

# Warnings are ok

 PYTHIA Warning in JunctionSplitting::SplitJunPairs: parallel junction state not allowed.  
 PYTHIA Warning in JunctionSplitting::CheckColours: Not possible to split junctions; making new colours  
 PYTHIA Error in BeamRemnants::add: failed to find physical colour state after colour reconnection  
 PYTHIA Error in BeamRemnants::setKinematics: kinematics construction failed  
 PYTHIA Warning in MultipartonInteractions::pTnext: weight above unity  
 PYTHIA Error in SimpleSpaceShower::pT2nearThreshold: stuck in loop  


Warning in <TROOT::Append>: Replacing existing TH1: h_all (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: h_pure (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c


In [7]:
getKstarCandidates(particles)
# isFinal("electron", "e")

NameError: name 'getKstarCandidates' is not defined

In [ ]:
particles

In [ ]:
parts

['electron', 'antielectron', 'gamma']